In [1]:
import os
from pathlib import Path
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers

path = '../mergedData/'

image_gen = ImageDataGenerator(validation_split=0.3)
train_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(224, 224),batch_size=64, class_mode='categorical', subset='training')
val_generator = image_gen.flow_from_directory(
    path + "Training", target_size=(224, 224),batch_size=64, class_mode='categorical', subset='validation')

test_generator = image_gen.flow_from_directory(
    path + "Test", target_size=(224, 224),batch_size=64, class_mode='categorical')


Found 27680 images belonging to 67 classes.
Found 11833 images belonging to 67 classes.
Found 16999 images belonging to 67 classes.


In [2]:
print(train_generator.class_indices)
print(len(train_generator.class_indices))

{'Apple': 0, 'Apricot': 1, 'Avocado': 2, 'Banana': 3, 'Beetroot': 4, 'Blueberry': 5, 'Cactus': 6, 'Cantaloupe': 7, 'Carambula': 8, 'Cauliflower': 9, 'Cherry': 10, 'Chestnut': 11, 'Clementine': 12, 'Cocos': 13, 'Corn': 14, 'Cucumber': 15, 'Dates': 16, 'Eggplant': 17, 'Fig': 18, 'Ginger': 19, 'Granadilla': 20, 'Grape': 21, 'Grapefruit': 22, 'Guava': 23, 'Hazelnut': 24, 'Huckleberry': 25, 'Kaki': 26, 'Kiwi': 27, 'Kohlrabi': 28, 'Kumquats': 29, 'Lemon': 30, 'Limes': 31, 'Lychee': 32, 'Mandarine': 33, 'Mango': 34, 'Mangostan': 35, 'Maracuja': 36, 'Melon': 37, 'Mulberry': 38, 'Nectarine': 39, 'Nut': 40, 'Onion': 41, 'Orange': 42, 'Papaya': 43, 'Passion': 44, 'Peach': 45, 'Pear': 46, 'Pepino': 47, 'Pepper': 48, 'Physalis': 49, 'Pineapple': 50, 'Pitahaya': 51, 'Plum': 52, 'Pomegranate': 53, 'Pomelo': 54, 'Potato': 55, 'Quince': 56, 'Rambutan': 57, 'Raspberry': 58, 'Redcurrant': 59, 'Salak': 60, 'Strawberry': 61, 'Tamarillo': 62, 'Tangelo': 63, 'Tomato': 64, 'Walnut': 65, 'Watermelon': 66}
67


In [3]:
#    Convolution
#    Non Linearity (ReLU)
#    Pooling or Sub Sampling
#    Classification (Fully Connected Layer)
#    Peamised põhilayerid

In [4]:
inputs = layers.Input(shape=(224, 224, 3))
#x = img_augmentation(inputs)
model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs, classes=len(train_generator.class_indices))

model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(len(train_generator.class_indices), activation="softmax", name="pred")(x)

model = tf.keras.Model(inputs, outputs)

In [5]:
strategy = tf.distribute.MirroredStrategy() #Ei tea kas vajalik või mis üldse teeb

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [6]:
seed = 1111

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 224, 224, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization[0][0]              
_______________________________________________________________________________________

In [7]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(train_generator, epochs=10, validation_data=val_generator, verbose=1)
results = model.evaluate(test_generator)
print(results)

Epoch 1/10
433/433 [==============================] - 1304s 3s/step - loss: 0.3297 - accuracy: 0.9294 - val_loss: 0.2466 - val_accuracy: 0.9412
Epoch 2/10
433/433 [==============================] - 1557s 4s/step - loss: 0.0255 - accuracy: 0.9968 - val_loss: 0.2225 - val_accuracy: 0.9463
Epoch 3/10
433/433 [==============================] - 1488s 3s/step - loss: 0.0129 - accuracy: 0.9984 - val_loss: 0.2061 - val_accuracy: 0.9516
Epoch 4/10
433/433 [==============================] - 1337s 3s/step - loss: 0.0084 - accuracy: 0.9987 - val_loss: 0.2082 - val_accuracy: 0.9544
Epoch 5/10
433/433 [==============================] - 1302s 3s/step - loss: 0.0051 - accuracy: 0.9994 - val_loss: 0.2040 - val_accuracy: 0.9555
Epoch 6/10
433/433 [==============================] - 1574s 4s/step - loss: 0.0047 - accuracy: 0.9993 - val_loss: 0.2339 - val_accuracy: 0.9477
Epoch 7/10
433/433 [==============================] - 1505s 3s/step - loss: 0.0045 - accuracy: 0.9992 - val_loss: 0.2505 - val_accuracy: